In [11]:
import sys
sys.path.append("R:\JoePriceResearch\Python\Anaconda3\pkgs")
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
os.chdir(r"C:\Users\josephwy\Documents")

# Bigfoot Sightings Data

I found a dataset that includes roughly 5000 bigfoot sightings from around the United States on https://data.world/timothyrenner/bfro-sightings-data. I downloaded it in a json format. 


## Upload Data

When I tried to upload the data, I was met with the following error: 

In [ ]:
with open(r"bfro_reports.json", "r") as f:
    bf_data = json.load(f)

## Incomplete Json

I spent quite a bit of time trying to determine the nature of the problem. After some research and inspection of the file, I found that it was not in proper json format. The file was lacking brackets around the data, and commas between each braced region. 

## Repair Json file

I wrote the following code to repair the data. I first saved the json as a text file. 
The code iterates line by line through the dataset, adding commas to seperate braced regions.
It then saves the document as a json. 
I then manually added brackets to the beginning and end of the json.

In [ ]:
with open(r"bfro_reports.txt", "r+") as f:
    mylist= list(f)
    newlist = []
    for line in mylist:
        line = line.strip() + ","
        print(line)
        newlist.append(line)
    f = str().join(newlist)
    with open(r"big_foot_data.json", "w") as bf:
        bf.write(f)

## Upload Round Two

It worked this time! 

In [ ]:
with open(r"big_foot_data.json", "r") as f:      
    bf_data = json.load(f)
    bf_df = pd.DataFrame(bf_data)    

## Data Overview

In [ ]:
 print(bf_df.head())

## Information of Interest

After scanning through the data, I decided that the most valuable information would be the locations and dates of the sightings. I did't have the time or the interest to read all of the first hand accounts. 

In [ ]:
print(bf_df.groupby('STATE').count())

In [ ]:
bf_df["STATE"].value_counts()
print(len(bf_df["STATE"].unique()))

## States Data in good shape

The states data all seemed pretty standardized and reasonable-- there were 49 states listed. This was pretty fortunate. 
Unfortunately, the Year data was a mess. 

In [ ]:
bf_df["YEAR"].value_counts()


## Cleaning the YEAR Data

There were 2 problems with the year data:
1. It was far from standardized and incredibly messy.
2. It was in string format and therefore difficult to treat chronologically

I decided to use regular expressions to replace all of the unusual dates I found. 
At the onset it seemed most efficient to just address each variation (such as "85-present") individually. I used the replace command and wrote verbatim regular expressions to match them. This proved to be a miserable experience and took far longer than I anticipated. I should have written a more general code. In any case, my regex replacement is shown below: 

In [ ]:
#cleanup YEAR 
#In cases where multiple dates are listed, I replace with the earliest.
#In the case of an ambiguous range, I replaced with the average
bf_df["YEAR"].replace(regex={r"^2000-2003" : "2002", "^1977 or 78": "1977",
     "^1977 or 78$": "1977", "^97-98$": "1977", "^mid 80's$": "1985", 
     "^\'01or \'02$": "2002", "^1980-1999$": "1990", "^04-05$": "2004",
     "^1997/1998$": "1997", "^95-96$": "1995", "^1991/1999$": "1991", 
     "^1947/48$": "1947", "In the 1980's$": "1985", "^1988-1989$" : "1988",
     "^92 or 93$": "1992", "1986- 2008$" : "1997", "^1980-82$" : "1981",
     "^1971 or 72$" : "1971", "2012`$" : "2012", "^Late1960's$" : "1968", 
     "^1956 or 1957$" : "1956", "^1987-88$" : "1987", "^1991-92$" : "1991",
     "^1974-1976$" : "1975", "^1984?$" : "1984", "^near 1983$" : "1983", 
     "^about 1960$" : "1960", "1976-1978$" : "1977", "^1981, 1982$" : "1981",
     "^Late 70s$" : "1978", "79, 80, 99$" : "1980", r"93/95$" : "1993", 
     "^1985 - '87$" : "1986", "72 or 73$" : "1972", "^1971-72$" : "1971",
     "^1998 or 19$" : "1998", "^1989?$" : "1989", "^Early 1980s$" : "^1980",
     "^1994-1997+" : "1994", "^198945/86$" : "1989", "^1994+" : "1994", 
     "^198941, 1982$" : "1982", "^1978-1990$" : "1978", "^2014-2016" : "2014",
     r"^198943/2010" : "1989", "^2005 2009" : "2005", "^198949 &2005" : "1989",
     "^1973 or 74" : "1973", "^1992-3" : "1992", "^1970-1972" : "1970", 
     "^Early 1990's" : "1992", "^198949" : "1989", "^198947" : "1989", 
     "^1978 thru 1982" : "1978", "^1973-'76" : "1973", "^19894[\d]" : "1989", 
     "^app. 1985" : "1985", "^2004-2005" : "2004", "^1959 or 60" : "1959", 
     "^1978-79" : "1978", "^2010/2011" : "2010", "^2008/2009" : "2008", 
     r"^1/5/1998" : "1998", "^1990 appro" : "1990", "^2005-06" : "2005", 
     "^2009-2010" : "2009", "^93-98" : "1993", "^71' or 72'" : "1971", 
     "^1930's" : "1930", "^1994\?" : "1994", "^1976 or 77" : "1976", "mid 1970's" : "1970", 
     "^Late 1980s" : "1988", "^1994[\d]+" : "1994", "^1961 1962" : "1961",
     "^1952 or 1953" : "1952", "^1976-2000-2008" : "1976", "^1989-86" : "1989",
     "^1979 or 80" : "1979", "^2000/2001" : "2000", "^93" : "1993",
     "^2000-2001" : "2000", "^1989,19943" : "1989", "^2001-2004" : "2001",
     "^07" : "2007", "^2002-2003" : "2002", "^119930" : "1993",
     "^119191919932" : "1993", "^202020202007" : "2007", "^119191919930" : "1993",
     "^1919191993" : "1993", "^2020202007" : "2007", "^1989 or '86" : "1989",
     "^11919930" : "1993", "^191993" : "1993", "^1970's-1980's" : "1970", 
     "^1994-2001" : "1994", "^1989 or 1987?" : "1987", 
     "^11919932" : "1993", "^191993" : "1993", "^1978 to 1980" : "1978", 
     "^est mid-70" : "1975", 
     "^About 1977" : "1977", "^" : "", "^" : "", "^" : "", 
     "^" : "", "^" : "", "^" : "", "^" : "", 
     "^" : "", "^" : "", "^" : "", "^" : "", 
     "^" : "", "^" : "", "^" : "", "^" : "",  }, inplace=True)


## Progress Check

I tried to see how many of the 400+ unique "YEAR" listings had been whittled down by my 
superhuman efforts. You can imagine my frustration to see the following:

In [ ]:
print(bf_df["YEAR"].unique())
print(len(bf_df["YEAR"].unique()))

In [ ]:
type(bf_df)

## Desperate times call for desperate measures

I was beyond jaded to find that I had only eliminated 100 of the faulty years.
In a fit of rage, I wrote the following code: 

In [ ]:
bf_df = bf_df.dropna(subset=["YEAR"])

In [ ]:
good_years = "^[\d]{4}$"
bf_df = bf_df[bf_df["YEAR"].str.match(good_years)]

In [ ]:
print(bf_df["YEAR"].unique())
print(len(bf_df["YEAR"].unique()))

In [ ]:
bf_df["YEAR"].value_counts()

In [ ]:
print(len(bf_df.index))

## YEAR Cleaning Continued

At the expense of roughly 800 entries (which is admittedly nontrivial, but also not as bad as wasting 4 more hours on bigfoot data cleaning,) I managed to clear out all those nasty YEAR entries. I rationalized that because their misentry was quasi-random and I still had plenty of other entries, this was an acceptable cleaning method. 

Then I converted the years into integers

In [ ]:
bf_df[['YEAR']] = bf_df[['YEAR']].apply(pd.to_numeric)

To verify that they were in fact integers, I took maxes and mins:


In [ ]:
print(bf_df["YEAR"].max())
print(bf_df["YEAR"].min())


## Data Visualization

Then I wanted to visualize bigfoot sightings by state per year. The first scatterplot was illegible:

In [ ]:
sns.set(style="whitegrid")

ax = sns.swarmplot(x="STATE", y="YEAR", data=bf_df)

So I decided to group the states by US Census Regions: Northeast, Midwest, South, and West. I did so by creating my own dataframe to merge with the bigfoot one. 

In [ ]:
with open(r"state_region.csv", "r") as j:
    state_df = pd.read_csv(j)

In [ ]:
state_df.head()

In [ ]:
bf_df = bf_df.merge(state_df, left_on='STATE', right_on='STATE', how='outer')

With the merge complete, I then played around with some new graphs:

In [ ]:
sns.set(rc={'figure.figsize':(12,9)})
ax = sns.swarmplot(x="REGION", y="YEAR", data=bf_df)

In [ ]:
sns.set(rc={'figure.figsize':(15,12)})
ax = sns.violinplot(x="REGION", y="YEAR", data=bf_df, inner=None)
ax = sns.swarmplot(x="REGION", y="YEAR", data=bf_df,color="white", edgecolor="gray")

In [ ]:
print(bf_df["REGION"].value_counts())

In [ ]:
print(bf_df["YEAR"].value_counts())

## Conclusions:

The Pacific is a hotspot for bigfoot sightings. I'm certain that he spent some time in the region, likely within the last 18 years. 

# Alchohol Consumption by State

I found the following data in a Washington Post article:
https://www.washingtonpost.com/news/wonk/wp/2016/12/24/where-the-heaviest-drinking-americans-live/?noredirect=on&utm_term=.4070d0f562d6

It gives the average monthly drinking rate per state for the years 2008-2009 and 2014-2015

## Reading in the Data

I scraped in the data from the Washington post website and parsed it using BeautifulSoup

In [ ]:
site_text = requests.get("https://www.washingtonpost.com/news/wonk/wp/2016/12/24/where-the-heaviest-drinking-americans-live/?noredirect=on&utm_term=.4070d0f562d6").text
soup = BeautifulSoup(site_text, 'html.parser')


Here is the prettified file:

In [ ]:
print(soup.prettify())

## Tabulating the Data

I tabulated the data using the pandas read_html function. It came out very nicely:

In [ ]:
alchohol_table = soup.find_all('table')[0]     
alch_df = pd.read_html(str(alchohol_table))[0]

In [ ]:
print(alch_df)

The data is very clean and simple and doesn't require any further preparation. For this reason, I played with it in some other ways. 

First, I renamed the column labels, since they are so lengthy

In [ ]:
alch_df = alch_df.rename(index=str, columns={"Monthly drinking rate, 2008-2009": "Drinking_Rate_08-09", "Monthly drinking rate, 2014-2015": "Drinking_Rate_14-15"})

In [ ]:
list(alch_df)

Next, I created a new variable that is the average of the drinking rates over the two periods (2008-2009 and 2014-2015). 
This way I have a more general drinking rate per state. 

In order to do so, I had to change the column type to int

In [ ]:
alch_df[['Drinking_Rate_08-09']] = alch_df[['Drinking_Rate_08-09']].apply(pd.to_numeric)

In [ ]:
alch_df['Avg_Drinking_Rate'] = (alch_df["Drinking_Rate_08-09"] + alch_df["Drinking_Rate_14-15"])/2

In [ ]:
alch_df

In [ ]:
alch_df = alch_df.merge(state_df, left_on='State', right_on='STATE', how='outer')

In [ ]:
bx = sns.swarmplot(x="State", y="Avg_Drinking_Rate", data=alch_df)

Again, the plot is pretty illegible due to the large number of states. Good thing I still had my State_region csv

In [ ]:
alch_df = alch_df.merge(state_df, left_on='State', right_on='STATE', how='outer')

In [ ]:
alch_df

In [ ]:
bx = sns.swarmplot(x="REGION_x", y="Avg_Drinking_Rate", data=alch_df)

In [ ]:
bf_df = bf_df.merge(alch_df, left_on='STATE', right_on='STATE_x', how='outer')

In [ ]:
cx = sns.swarmplot(x="REGION_x", y="Avg_Drinking_Rate", data=alch_df)

In [ ]:
bf_df['freq'] = bf_df.groupby('STATE')['STATE'].transform('count')

In [ ]:
bf_df['freq'].describe()

In [ ]:
dx = sns.swarmplot(x="freq", y="Avg_Drinking_Rate", data=bf_df)